<a href="https://colab.research.google.com/github/Al3jandraMtz/Proyecto4_DataLab/blob/main/P4_Varios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('/content/Data-Set-Sentimiento1.csv')

# Mostrar las primeras filas del DataFrame para verificar los nombres de las columnas
print(df.head())
print(df.columns)


   product_id                                       product_name  \
0  B08XLR6DSB  akiara - Makes life easy Electric Handy Sewing...   
1  B0978V2CP6  Cubetek 3 in 1 LCD Display V5.0 Bluetooth Tran...   
2  B07WKBD37W  ESnipe Mart Worldwide Travel Adapter with Buil...   
3  B0B2RBP83P  Lenovo IdeaPad 3 11th Gen Intel Core i3 15.6" ...   
4  B0BLV1GNLN  WZATCO Pixel | Portable LED Projector | Native...   

              category_1              category_2                  category_3  \
0           Home&Kitchen  Kitchen&HomeAppliances  SewingMachines&Accessories   
1            Electronics    HomeTheater,TV&Video      AVReceivers&Amplifiers   
2        HomeImprovement              Electrical      Adapters&Multi-Outlets   
3  Computers&Accessories                 Laptops          TraditionalLaptops   
4            Electronics    HomeTheater,TV&Video                  Projectors   

                  category_4 discounted_price  actual_price  \
0  Sewing&EmbroideryMachines              721  

In [7]:
#Segmentación de los sentimientos
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('/content/Data-Set-Sentimiento1.csv')

# Segmentar el sentimiento en positivo, neutro y negativo
def segment_sentiment(score):
    if score > 0.1:
        return 'positivo'
    elif score < -0.1:
        return 'negativo'
    else:
        return 'neutro'

# Aplicar la función para crear una nueva columna de 'sentiment_category'
df['sentiment_category'] = df['sentiment'].apply(segment_sentiment)

# Guardar el DataFrame en un archivo CSV
df.to_csv('/content/sentiment_segmented.csv', index=False)

print("Archivo CSV guardado con éxito.")


Archivo CSV guardado con éxito.


In [8]:

#Analisis de Sentimientos acomodo de palabras positivas/negativas
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter

# Descargar el recurso vader_lexicon
nltk.download('vader_lexicon')

# Inicializar el analizador de sentimiento
sia = SentimentIntensityAnalyzer()

# Cargar el DataFrame
df = pd.read_csv('/content/Data-Set-Sentimiento1.csv')

# Función para analizar el sentimiento y extraer palabras
def extract_sentiment_words(text):
    sentiment_words = {'positive': [], 'negative': []}
    # Analizar el sentimiento de cada palabra en el texto
    for word in text.split():
        sentiment = sia.polarity_scores(word)
        if sentiment['compound'] >= 0.05:  # Umbral para positivo
            sentiment_words['positive'].append(word)
        elif sentiment['compound'] <= -0.05:  # Umbral para negativo
            sentiment_words['negative'].append(word)
    return sentiment_words

# Aplicar la función a la columna cleaned_review_content
df['sentiment_words'] = df['cleaned_review_content'].apply(extract_sentiment_words)

# Contar las palabras más comunes en positivas y negativas
positive_words = Counter([word for words in df['sentiment_words'].apply(lambda x: x['positive']) for word in words])
negative_words = Counter([word for words in df['sentiment_words'].apply(lambda x: x['negative']) for word in words])

# Crear DataFrame para las Top Positive Words
df_positive = pd.DataFrame(positive_words.most_common(20), columns=['Word', 'Count'])
df_positive['Sentiment'] = 'Positive'

# Crear DataFrame para las Top Negative Words
df_negative = pd.DataFrame(negative_words.most_common(20), columns=['Word', 'Count'])
df_negative['Sentiment'] = 'Negative'

# Combinar ambos DataFrames
df_top_words = pd.concat([df_positive, df_negative])

# Guardar los resultados en un nuevo CSV en el directorio actual
df_top_words.to_csv('top_sentiment_words.csv', index=False)

# Mostrar el DataFrame resultante
print(df_top_words)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


            Word  Count Sentiment
0           good   4179  Positive
1           like   1065  Positive
2           easy    743  Positive
3           well    653  Positive
4         better    548  Positive
5          great    533  Positive
6           best    521  Positive
7           nice    435  Positive
8         amazon    405  Positive
9           fine    399  Positive
10         value    329  Positive
11         worth    327  Positive
12          want    263  Positive
13            ok    232  Positive
14      original    228  Positive
15     recommend    215  Positive
16        easily    208  Positive
17       support    206  Positive
18        useful    187  Positive
19         happy    187  Positive
0             no    987  Negative
1        problem    290  Negative
2            low    255  Negative
3            bad    197  Negative
4           hard    137  Negative
5        stopped     93  Negative
6           poor     87  Negative
7      difficult     80  Negative
8            l

In [9]:
import pandas as pd

# Cargar el archivo CSV con tus datos
df = pd.read_csv('/content/Data-Set-Sentimiento1.csv')

# Definir qué es una calificación baja (por ejemplo, calificación < 3)
low_rating_threshold = 3

# Crear una columna 'is_low_rating' que indica si la calificación es baja
df['is_low_rating'] = df['adjusted_rating'] < low_rating_threshold

# Agrupar por categoría y nivel de descuento, y calcular el riesgo relativo
# El riesgo relativo se define como la proporción de calificaciones bajas en cada grupo
df_grouped = df.groupby(['category_1', 'discount_percentage']).apply(
    lambda x: x['is_low_rating'].mean()
).reset_index(name='low_rating_risk')

# Guardar el DataFrame en un archivo CSV
df_grouped.to_csv('/content/risk_relative_discount_category.csv', index=False)

print("Archivo CSV guardado con éxito para su uso en Tableau.")


Archivo CSV guardado con éxito para su uso en Tableau.


In [10]:
import pandas as pd

# Cargar el archivo CSV con tus datos
df = pd.read_csv('/content/Data-Set-Sentimiento1.csv')

# Palabras clave relacionadas con durabilidad
durability_keywords = ['durabilidad', 'duradero', 'resistente', 'longevidad', 'durable']

# Crear una columna 'mentions_durability' para identificar menciones de durabilidad en las reseñas
df['mentions_durability'] = df['cleaned_review_content'].apply(
    lambda review: any(keyword in review.lower() for keyword in durability_keywords)
)

# Calcular la satisfacción promedio (por ejemplo, 'adjusted_rating') en función de menciones de durabilidad
durability_impact = df.groupby('mentions_durability')['adjusted_rating'].mean().reset_index()

# Renombrar las columnas para claridad
durability_impact.columns = ['mentions_durability', 'average_satisfaction']

# Guardar el DataFrame en un archivo CSV
durability_impact.to_csv('/content/durability_impact_satisfaction.csv', index=False)

print("Archivo CSV guardado con éxito para su uso en herramientas de visualización.")


Archivo CSV guardado con éxito para su uso en herramientas de visualización.


In [11]:
import pandas as pd

# Cargar el archivo CSV con tus datos
df = pd.read_csv('/content/Data-Set-Sentimiento1.csv')

# Verificar las categorías únicas en 'category_1'
print("Categorías únicas en 'category_1':", df['category_1'].unique())

# Asegurarnos de que las columnas necesarias están presentes
print("Columnas disponibles en el DataFrame:", df.columns)

# Filtrar las categorías "Toys & Games" y "Home & Kitchen"
categories_of_interest = ['Toys & Games', 'Home & Kitchen']
df_filtered = df[df['category_1'].isin(categories_of_interest)]

# Verificar el tamaño del DataFrame filtrado
print(f"Tamaño del DataFrame después del filtrado: {df_filtered.shape}")

if df_filtered.empty:
    print("No se encontraron datos para las categorías especificadas.")
else:
    # Calcular la calificación promedio por categoría
    average_rating_per_category = df_filtered.groupby('category_1')['adjusted_rating'].mean().reset_index()

    # Mostrar las calificaciones promedio calculadas
    print("Calificación promedio por categoría:")
    print(average_rating_per_category)

    # Calcular la tendencia de calificaciones promedio por categoría
    trend_data = df_filtered.groupby(['category_1']).agg(
        average_rating=('adjusted_rating', 'mean')).reset_index()

    # Ordenar los datos por categoría
    trend_data = trend_data.sort_values(by=['category_1'])

    # Guardar el DataFrame en un archivo CSV para su uso en herramientas de visualización como Tableau
    trend_data.to_csv('/content/ratings_trend_by_category.csv', index=False)

    print("Archivo CSV guardado con éxito para su uso en herramientas de visualización.")


Categorías únicas en 'category_1': ['Home&Kitchen' 'Electronics' 'HomeImprovement' 'Computers&Accessories'
 'MusicalInstruments' 'Health&PersonalCare' 'Car&Motorbike' 'Toys&Games'
 'OfficeProducts']
Columnas disponibles en el DataFrame: Index(['product_id', 'product_name', 'category_1', 'category_2', 'category_3',
       'category_4', 'discounted_price', 'actual_price', 'discount_percentage',
       'about_product', 'user_id', 'user_name', 'user_id_count', 'review_id',
       'review_title', 'cleaned_review_content', 'rating_count',
       'adjusted_rating', 'tokenized_review', 'filtered_tokens',
       'stemmed_tokens', 'sentiment'],
      dtype='object')
Tamaño del DataFrame después del filtrado: (0, 22)
No se encontraron datos para las categorías especificadas.


In [12]:
#satisfaccion por categoria
import pandas as pd

# Cargar el archivo CSV con tus datos
df = pd.read_csv('/content/Data-Set-Sentimiento1.csv')

# Verificar que las columnas necesarias están presentes
if 'category_1' not in df.columns or 'adjusted_rating' not in df.columns:
    raise ValueError("Las columnas 'category_1' y 'adjusted_rating' deben estar presentes en el DataFrame.")

# Calcular la calificación promedio por categoría
average_rating_per_category = df.groupby('category_1')['adjusted_rating'].mean().reset_index()

# Definir umbrales para alta y baja satisfacción
threshold_high = 4.0  # Ajusta este valor según tus criterios
threshold_low = 2.5   # Ajusta este valor según tus criterios

# Clasificar las categorías según el nivel de satisfacción
def classify_satisfaction(rating):
    if rating >= threshold_high:
        return 'alta satisfacción'
    elif rating <= threshold_low:
        return 'baja satisfacción'
    else:
        return 'satisfacción media'

average_rating_per_category['satisfaction_level'] = average_rating_per_category['adjusted_rating'].apply(classify_satisfaction)

# Mostrar las categorías clasificadas
print("Categorías clasificadas por nivel de satisfacción:")
print(average_rating_per_category)

# Guardar el DataFrame en un archivo CSV para uso futurohttps://www.google.com.mx/
average_rating_per_category.to_csv('/content/category_satisfaction_levels.csv', index=False)

print("Archivo CSV guardado con éxito para su uso en intervenciones.")


Categorías clasificadas por nivel de satisfacción:
              category_1  adjusted_rating  satisfaction_level
0          Car&Motorbike         3.800000  satisfacción media
1  Computers&Accessories         4.153867   alta satisfacción
2            Electronics         4.077755   alta satisfacción
3    Health&PersonalCare         4.000000   alta satisfacción
4           Home&Kitchen         4.040402   alta satisfacción
5        HomeImprovement         4.250000   alta satisfacción
6     MusicalInstruments         3.900000  satisfacción media
7         OfficeProducts         4.309677   alta satisfacción
8             Toys&Games         4.300000   alta satisfacción
Archivo CSV guardado con éxito para su uso en intervenciones.
